# Medical Image Analysis

In [ ]:
! pip install numpy
! pip install matplotlib
! pip install wfdb
! pip install scipy
! pip install ipywidgets

In [ ]:
import wfdb
import matplotlib as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import os

## data read

In [ ]:
# 현재 경로 불러오기 = os.getcwd()
path = os.getcwd()
os.chdir(path+ "\\training2017")
print(os.getcwd())

In [ ]:
csv='REFERENCE.csv'
data = pd.read_csv(csv, header=None, names=['ID', 'Label'])

In [ ]:
def ecg_read(patient_id):
    record = wfdb.rdrecord(patient_id) 
    ecg_signal = record.p_signal[:,0]  
    fs = record.fs
    return ecg_signal, fs

### 해당 Annotation  id 저장

In [ ]:
A =[] # Atrial Fibrillation
N= [] # Normal Sinus rhythm
O = [] # Other rhythm
I = [] # Noisy

for id, lb in zip (data['ID'], data['Label'] ): #v2 data
         
         if lb =='A':
            A.append(id)
            
         elif lb =='N':
            N.append(id)
         
         elif lb =='O':
            O.append(id)
        
         elif lb =='~':
            I.append(id)   

### dictionary 형태로 저장

In [ ]:
dic = {'Atrial Fibrillation': A, 'Normal Sinus Rhythm': N, 'Other Rhythm': O, 'Noisy': I}

## Calculate BPM

In [ ]:
import numpy as np
from scipy.signal import find_peaks

def calculate_bpm(ecg_signal, fs):
    """
    Calculate the beats per minute (BPM) from an ECG signal.

    :param ecg_signal: ECG signal (1D numpy array).
    :param fs: Sampling frequency of the ECG signal.
    :return: Calculated BPM.
    """
    # R파 감지
    peaks, _ = find_peaks(ecg_signal, distance=fs*0.6)  # R파는 보통 1분에 60~100회 사이

    # R파 간의 간격 계산
    rr_intervals = np.diff(peaks) / fs  # 초 단위로 변환

    # 평균 RR 간격을 기반으로 BPM 계산
    mean_rr = np.mean(rr_intervals)
    bpm = 60 / mean_rr  # 분당 박동수

    return bpm

## ECG plot

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

dic = {'Atrial Fibrillation': A, 'Normal Sinus Rhythm': N, 'Other Rhythm': O, 'Noisy': I}


def update_plot(patient_id, time_range, ecglabel):
    ecg_signal, fs = ecg_read(patient_id)
    start, end = time_range
    start_index = int(start * fs)
    end_index = int(end * fs)
    
    plt.figure(figsize=(15, 3))
    plt.plot(np.linspace(start, end, end_index-start_index), ecg_signal[start_index:end_index])
    plt.title(f'{ecglabel}: Patient {patient_id} - ECG Signal')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.show()




# 위젯 정의
label_dropdown = widgets.Dropdown(options=list(dic.keys()), description='Select ECG Label:', width='300px')
patient_dropdown = widgets.Dropdown(options=[], description='Select Patient ID:',width='300px')
time_slider = widgets.IntRangeSlider(value=[0, 30], min=0, max=30, step=1, description='Time Range (s):', continuous_update=False)

# 드롭다운 위젯들의 옵션을 업데이트하는 함수
def update_patient_dropdown(*args):
    patient_dropdown.options = dic[label_dropdown.value]

# 레이블 선택에 따라 환자 ID 드롭다운 옵션을 업데이트하는 함수
label_dropdown.observe(update_patient_dropdown, 'value')



#환자 ID가 변경될 때 호출되는 함수
def on_patient_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        clear_output(wait=True)  # 기존 출력 내용을 지우기
        display(widgets.HBox([label_dropdown, patient_dropdown])) # 새로운 이미지 생성시 LABEL, ID 띄우기
        patient_id = change['new']
        ecg_signal, fs = ecg_read(patient_id)
        max_time = len(ecg_signal) / fs
        time_slider.max = max_time
        time_slider.value = [0, max_time]  # Reset to full range
        update_plot(patient_id, time_slider.value, label_dropdown.value)

        
       

        

patient_dropdown.observe(on_patient_dropdown_change, 'value')

# 모든 위젯을 표시
display(widgets.HBox([label_dropdown, patient_dropdown]))



In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import interact
import matplotlib.pyplot as plt


dic = {'Atrial Fibrillation': A, 'Normal Sinus Rhythm': N, 'Other Rhythm': O, 'Noisy': I}


def update_plot(patient_id, time_range):
        ecg_signal, fs = ecg_read(patient_id)
        start, end = time_range
        start_index = int(start * fs)
        end_index = int(end * fs)
        
        plt.figure(figsize=(15, 3))
        plt.plot(np.linspace(start, end, end_index-start_index), ecg_signal[start_index:end_index])
        plt.title(f'{label_dropdown.value}: Patient {patient_id} - ECG Signal')
        plt.xlabel('Time (s)')
        plt.ylabel('Amplitude')
        plt.show()
# 위젯 정의
label_dropdown = widgets.Dropdown(options=list(dic.keys()), description='Label:', width = '500px')
patient_dropdown = widgets.Dropdown(options=[], description='ID:', width = '300px')
time_slider = widgets.IntRangeSlider(value=[0, 30], min=0, max=30, step=1, description='Time Range (s):', continuous_update=False)


# 드롭다운 위젯들의 옵션을 업데이트하는 함수
def update_patient_dropdown(*args):
    patient_dropdown.options = dic[label_dropdown.value]

label_dropdown.observe(update_patient_dropdown, 'value')

# 환자 ID가 변경될 때 호출되는 함수
def on_patient_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        clear_output(wait=True)  # Clear the output below the widget
        patient_id = change['new']
        ecg_signal, fs = ecg_read(patient_id)
        max_time = len(ecg_signal) / fs
        time_slider.max = max_time
        time_slider.value = [0, max_time]  # Reset to full range
        # 현재 위젯을 clear하고 새로운 위젯을 display
        clear_output(wait=True)
        display(widgets.HBox([label_dropdown, patient_dropdown]))
        # interact 함수를 사용하여 슬라이더와 update_plot 함수를 연결
        interact(update_plot, patient_id=widgets.fixed(patient_id), time_range=time_slider)

patient_dropdown.observe(on_patient_dropdown_change, 'value')

# Display the widgets
display(widgets.HBox([label_dropdown, patient_dropdown]))


## Time distribution

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# ECG 데이터 읽기 및 시간대별 분류
time_categories = {'>60s': [], '51-60s': [], '41-50s': [], '31-40s': [], '21-30s': [], '<=20s': []}
time_list = []

# ECG 데이터 읽기  
for id, lb in zip (data['ID'], data['Label'] ):
    record = wfdb.rdrecord(id)
    ecg_signal = record.p_signal[:,0]
    fs = record.fs
    time = len(ecg_signal) / fs
    time_list.append(time)

    # 시간대별 ID 분류
    if time > 60:
        time_categories['>60s'].append(id)
    elif 50 < time <= 60:
        time_categories['51-60s'].append(id)
    elif 40 < time <= 50:
        time_categories['41-50s'].append(id)
    elif 30 < time <= 40:
        time_categories['31-40s'].append(id)
    elif 20 < time <= 30:
        time_categories['21-30s'].append(id)
    else:
        time_categories['<=20s'].append(id)



In [ ]:
# 히스토그램 생성
plt.hist(time_list, bins=20, edgecolor='black')
plt.title('Distribution of ECG Signal Lengths')
plt.xlabel('Time in seconds')
plt.ylabel('Number of ECG Signals')
plt.show()

# 각 시간대별 ID 개수 DataFrame 생성
time_counts = {k: len(v) for k, v in time_categories.items()}
df_time_counts = pd.DataFrame(list(time_counts.items()), columns=['Time Category', 'Number of IDs'])
df_time_counts


## Label Information

In [ ]:
import pandas as pd

A =[] # Atrial Fibrillation
N= [] # Normal Sinus rhythm
O = [] # Other rhythm
I = [] # Noisy

for id, lb in zip (data['ID'], data['Label'] ): #v2 data
         
         if lb =='A':
            A.append(id)
            
         elif lb =='N':
            N.append(id)
         
         elif lb =='O':
            O.append(id)
        
         elif lb =='~':
            I.append(id)   

# 주어진 데이터에서 각 레이블 별 ID 수 계산
label_counts = {'Atrial Fibrillation': len(A),
                'Normal Sinus Rhythm': len(N),
                'Other Rhythm': len(O),
                'Noisy': len(I)}

# 통계 DataFrame 생성
df_label_counts = pd.DataFrame((label_counts.items()), columns=['ECG Label', 'Number of IDs'])


# 통계 출력
print(df_label_counts)

# 각 레이블에 대한 비율 계산
total_ids = sum(label_counts.values())
df_label_counts['Percentage'] = (df_label_counts['Number of IDs'] / total_ids) * 100

# 비율을 포함한 통계 출력
print(df_label_counts)




## Scalogram

In [ ]:
import pywt

def plot_scalogram(patient_id, ecg_signal, fs, cmap,  wavelet_name='cmor',scale =64, reverse= False):
    """
    Plot the scalogram of an ECG signal.

    :param ecg_signal: ECG signal (1D numpy array).
    :param fs: Sampling frequency of the ECG signal.
    :param scales: Scales to be used for the Continuous Wavelet Transform.
    :param wavelet_name: Name of the wavelet to be used.
    """
    # 웨이블릿 스케일
    widths = np.arange(1,scale)
    # 연속 웨이블릿 변환 수행
    wavelet_coeffs, freqs = pywt.cwt(ecg_signal, widths, wavelet=wavelet_name, sampling_period= 1/fs)
    end= int(len(ecg_signal) / fs) *10
    # print(end)
    # print(wavelet_coeffs.shape)
    # # Plotting the scalogram
    plt.figure()
    # 성분 반전 표시
    if reverse:
        plt.imshow((np.abs(np.flipud(wavelet_coeffs[:, :]))), cmap= cmap, aspect='auto', extent=[0, len(ecg_signal) / fs, 1, max(widths)])
    else:
        plt.imshow((np.abs(wavelet_coeffs[:, :])), cmap= cmap, aspect='auto', extent=[0, len(ecg_signal) / fs, 1, max(widths)])
    plt.ylabel('Scale')
    plt.xlabel('Time (seconds)')
    plt.title(f'Scalogram {patient_id} scale:{scale}')
    # plt.colorbar(label='Magnitude')
   

In [ ]:
ecg_signal,fs = ecg_read('A00001')
# PT Algorithm적용
# pt_tompkins = Pan_tompkins(ecg_signal,fs).fit()
# pt_signal = pt_tompkins

plot_scalogram('A00001', ecg_signal, fs, wavelet_name= 'morl', cmap= 'jet')
plot_scalogram('A00001', ecg_signal, fs, wavelet_name= 'morl', cmap= 'turbo', scale= 4, reverse= False)
plot_scalogram('A00001', ecg_signal, fs, wavelet_name= 'morl', cmap= 'turbo', scale= 8, reverse= False)
plot_scalogram('A00001', ecg_signal, fs, wavelet_name= 'morl', cmap= 'turbo', scale= 16, reverse= False)
plot_scalogram('A00001', ecg_signal, fs, wavelet_name= 'morl', cmap= 'turbo', scale= 32, reverse= False)
plot_scalogram('A00001', ecg_signal, fs, wavelet_name= 'morl', cmap= 'turbo', scale= 64, reverse= False)
plot_scalogram('A00001', ecg_signal, fs, wavelet_name= 'morl', cmap= 'turbo', scale= 128, reverse= False)
plot_scalogram('A00001', ecg_signal, fs, wavelet_name= 'morl', cmap= 'turbo', scale= 256, reverse= False)
plot_scalogram('A00001', ecg_signal, fs, wavelet_name= 'morl', cmap= 'turbo', scale= 512, reverse= False)
plot_scalogram('A00001', ecg_signal, fs, wavelet_name= 'morl', cmap= 'turbo', scale= 1024, reverse= False)

## Cardio Petal(All Sample)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pywt
from matplotlib.colors import LinearSegmentedColormap
# Define the colors for cherry blossom - pale pink to white, with hints of deeper pink



# Read the ECG signal and get the sampling frequency
# ecg_signal, fs = ecg_read('A00017')
# ecg_signal, fs = ecg_read('A00001')
ecg_signal,fs =ecg_read('A01246')
print('bpm: '+str(calculate_bpm(ecg_signal, fs)))
# Define the wavelet type and scales
wavelet = 'morl'  # Morlet wavelet
widths = np.arange(1, 64)  # Scales for the Morlet wavelet

# Perform the continuous wavelet transform
wavelet_coeffs, freqs = pywt.cwt(ecg_signal, widths, wavelet=wavelet, sampling_period= 1/fs)



# Calculate the time axis for the scalogram
times = np.linspace(0, len(ecg_signal) / fs, wavelet_coeffs.shape[1]) 
end= int(len(ecg_signal) / fs) 


#  Create a meshgrid for polar coordinates
# theta = 2 * np.pi * times / max(times)  # 해당 ecg 외에 추가된 정보가 포함되어 있다. 
theta = (2 * np.pi * times / end)  # Normalized time to [0, 2*pi]
rho = np.linspace(0.01, max(widths), len(widths))

T, R = np.meshgrid(theta, rho)


# Plot the scalogram in polar coordinates
plt.figure(figsize=(8, 8))
ax = plt.subplot(111, polar=True)
c = ax.pcolormesh(T, R, np.abs(np.flipud(wavelet_coeffs)), shading='gouraud', cmap='turbo')
# c = ax.pcolormesh(T, R, np.abs(wavelet_coeffs), shading='gouraud', cmap='turbo')

# Set the theta direction (0 at the top)
# ax.set_theta_zero_location('N')
# ax.set_theta_direction(-1)



# Set labels and title
ax.set_xlabel("Time (seconds)")
ax.set_ylabel("Scales")
ax.set_title("Noise Rythm[30s]: A01246")
ax.axis('off')

# Add colorbar
# plt.colorbar(c, ax=ax, pad=0.1, label='Magnitude')

plt.show()


## Beats plot
https://github.com/berndporr/py-ecg-detectors

In [ ]:
! pip install py-ecg-detectors

### r-peak detection

In [ ]:
import matplotlib.pyplot as plt
from ecgdetectors import Detectors

def plot_ecg_with_rpeaks(ecg_signal, r_peaks, fs):
    plt.figure(figsize=(15, 4))
    plt.plot(ecg_signal, label='ECG Signal')
    
    # R파 위치에 마커 표시
    for r_peak in r_peaks:
        plt.plot(r_peak, ecg_signal[r_peak], 'ro')
    
    plt.xlabel('Samples')
    plt.ylabel('Amplitude')
    plt.title('ECG Signal with R-Peaks')
    plt.legend()
    plt.show()

# ECG 신호와 R파 위치 시각화
ecg_signal, fs = ecg_read('A00077')
detectors = Detectors(fs)
r_peaks = detectors.pan_tompkins_detector(ecg_signal)
plot_ecg_with_rpeaks(ecg_signal, r_peaks, fs)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ecgdetectors import Detectors

def correct_r_peak_delays(ecg_signal, detected_peaks, fs):
    corrected_peaks = []
    for peak in detected_peaks:
        # 각각의 감지된 R-피크 주변에서 실제 최대값을 찾습니다.
        # 예를 들어, 주변 10개 샘플 내에서 최대값을 찾습니다.
        search_radius = 10  # 샘플 수
        search_region = ecg_signal[max(0, peak - search_radius):min(len(ecg_signal), peak + search_radius)]
        max_index = np.argmax(search_region)
        corrected_peak = peak - search_radius + max_index
        corrected_peaks.append(corrected_peak)
    return corrected_peaks

def plot_ecg_with_rpeaks(ecg_signal, r_peaks, fs):
    # R-피크에 지연이 있는 경우 이를 수정합니다.
    corrected_r_peaks = correct_r_peak_delays(ecg_signal, r_peaks, fs)
    
    plt.figure(figsize=(15, 4))
    plt.plot(ecg_signal, label='ECG Signal')
    plt.plot(r_peaks, ecg_signal[r_peaks], 'ro', label='Detected R-Peaks')
    plt.plot(corrected_r_peaks, ecg_signal[corrected_r_peaks], 'gx', label='Corrected R-Peaks')
    
    plt.xlabel('Samples')
    plt.ylabel('Amplitude')
    plt.title('ECG Signal with R-Peaks')
    plt.legend()
    plt.show()

# ECG 신호와 R파 위치 시각화
ecg_signal, fs = ecg_read('A00001')  # ECG 데이터를 로드하는 함수
detectors = Detectors(fs)
r_peaks = detectors.pan_tompkins_detector(ecg_signal)
plot_ecg_with_rpeaks(ecg_signal, r_peaks, fs)


### Segmentation

In [ ]:
import matplotlib.pyplot as plt
from ecgdetectors import Detectors

def plot_segmented_beats(ecg_signal, r_peaks, fs, window=0.5):
    # window 파라미터는 R파 주변으로 얼마나 많은 신호를 포함할 것인지를 결정합니다.
    # 여기서 window는 R파를 중심으로 전후 0.2초 동안의 데이터를 포함합니다.
    
    num_beats = len(r_peaks)
    plt.figure(figsize=(7, num_beats*2))  # 각 beat에 2 인치의 높이를 할당합니다.
    
    for i, r_peak in enumerate(r_peaks):
        # 각 R파의 인덱스에 대해 앞뒤로 window초 만큼의 데이터를 잘라냅니다.
        start_index = max(r_peak - int(window * fs), 0)
        end_index = min(r_peak + int(window * fs), len(ecg_signal))
        
        # 세그먼트 추출
        segment = ecg_signal[start_index:end_index]
       
        
        # 세그먼트 플롯
        plt.subplot(num_beats, 1, i + 1)
        plt.plot(segment)
        plt.title(f'Beat {i+1}')
        plt.xlabel('Samples')
        plt.ylabel('Amplitude')
        

    plt.tight_layout()
    plt.show()

# ECG 신호 로드 및 R파 검출
ecg_signal, fs = ecg_read('A00001')  # 여기서 ecg_read 함수는 ECG 데이터를 로드하는 사용자 정의 함수입니다.
detectors = Detectors(fs)
r_peaks = detectors.pan_tompkins_detector(ecg_signal)

print(len(r_peaks))

# 각 심장 박동을 세그먼트로 나누어 시각화
plot_segmented_beats(ecg_signal, r_peaks, fs)


### Plot cardio petal (1QRS)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pywt
import wfdb
from ecgdetectors import Detectors


def correct_r_peak_delays(ecg_signal, detected_peaks, fs):
    corrected_peaks = []
    for peak in detected_peaks:
        search_radius = 10
        search_region = ecg_signal[max(0, peak - search_radius):min(len(ecg_signal), peak + search_radius)]
        max_index = np.argmax(search_region)
        corrected_peak = peak - search_radius + max_index
        corrected_peaks.append(corrected_peak)
    return corrected_peaks

def plot_segmented_beats(ecg_signal, r_peaks, fs, window=1):
    wavelet = 'morl'
    num_beats = len(r_peaks)
    
    for i, r_peak in enumerate(r_peaks):
        start_index = max(0, r_peak - int(window * fs / 2))
        end_index = min(len(ecg_signal), r_peak + int(window * fs / 2))
        segment = ecg_signal[start_index:end_index]

        # 시각화 부분
        fig, axs = plt.subplots(1, 3, figsize=(15, 3))  # 1x3 subplot 준비

        # 첫 번째 서브플롯: 원본 ECG 세그먼트
        axs[0].plot(segment)
        axs[0].set_title(f'ECG Segment Around R-Peak {i+1}')
        axs[0].set_xlabel('Samples')
        axs[0].set_ylabel('Amplitude')

        # 두 번째 서브플롯: 스케일로그램
        scales = np.arange(1, 64)
        wavelet_coeffs, freqs = pywt.cwt(segment, scales, wavelet, sampling_period=1/fs)
        axs[1].imshow(np.abs(wavelet_coeffs), extent=[0, window, 1, max(scales)], cmap='turbo', aspect='auto', origin='lower')
        axs[1].set_title('Scalogram')
        axs[1].set_xlabel('Time (s)')
        axs[1].set_ylabel('Scales')

        # 세 번째 서브플롯: 스케일로그램을 극좌표계로 변환하여 플롯
        times = np.linspace(0, len(segment) / fs, len(segment))
        theta = 2 * np.pi * times / times[-1]
        rho = np.linspace(0.1, max(scales), len(scales))
        T, R = np.meshgrid(theta, rho)
        axs[2] = plt.subplot(1, 3, 3, polar=True)
        c = axs[2].pcolormesh(T, R, np.flipud(np.abs(wavelet_coeffs)), shading='auto', cmap='turbo')
        axs[2].set_title('Polar Scalogram')
        
        # 극좌표 축 제거
        axs[2].set_xticks([])
        axs[2].set_yticks([])
        axs[2].set_xticklabels([])
        axs[2].set_yticklabels([])
        axs[2].spines['polar'].set_visible(False)

        plt.tight_layout()
        plt.show()

# ECG 데이터와 R파 위치 로드
ecg_signal, fs = ecg_read('A00004')  # 'ecg_read' 함수가 ECG 데이터를 읽어오는 함수라고 가정
detectors = Detectors(fs)
r_peaks = detectors.pan_tompkins_detector(ecg_signal)
# 각 심장 박동의 세그먼트에 대해 스케일로그램 시각화
plot_segmented_beats(ecg_signal, r_peaks, fs)



### Sampling rate 600

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pywt

def plot_segmented_beats(ecg_signal, fs, segment_length=600):
    wavelet = 'morl'

    for start_index in range(0, len(ecg_signal), segment_length):
        end_index = min(start_index + segment_length, len(ecg_signal))
        segment = ecg_signal[start_index:end_index]

        # Plot the ECG segment
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 3, 1)
        plt.plot(segment)
        plt.title(f'ECG Segment {start_index} to {end_index}')
        plt.xlabel('Samples')
        plt.ylabel('Amplitude')

        # Calculate and plot the scalogram
        scales = np.arange(1, 64)
        wavelet_coeffs, freqs = pywt.cwt(segment, scales, wavelet, sampling_period=1/fs)
        
        plt.subplot(1, 3, 2)
        plt.imshow(np.abs(wavelet_coeffs), extent=[0, segment_length / fs, 1, max(scales)], cmap='turbo', aspect='auto', origin='lower')
        plt.title('Scalogram')
        plt.xlabel('Time (s)')
        plt.ylabel('Scales')

        # Calculate and plot the polar scalogram
        times = np.linspace(0, segment_length / fs, end_index - start_index)
        theta = 2 * np.pi * times / times[-1]
        rho = np.linspace(0.1, max(scales), len(scales))
        T, R = np.meshgrid(theta, rho)
        
        plt.subplot(1, 3, 3, polar=True)
        plt.pcolormesh(T, R, np.abs(np.flipud(wavelet_coeffs)), shading='auto', cmap='turbo')
        plt.title(f"Noise-{ID}")
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()

# Assuming ecg_read function is defined earlier in the code
# Load the ECG signal
ID = 'A01246'
ecg_signal, fs = ecg_read(ID)  # Replace with the correct function to read ECG data

# Visualize the ECG signal segments and their corresponding scalograms
plot_segmented_beats(ecg_signal, fs)


In [ ]:
fs

## Segmentation UI

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pywt
import wfdb
from ipywidgets import interactive, IntSlider, fixed

# Define the wavelet
wavelet = 'morl'

# ECG 데이터 로드 함수 (가정)
def ecg_read(record_name):
    record = wfdb.rdrecord(record_name)
    ecg_signal = record.p_signal[:,0]
    fs = record.fs
    return ecg_signal, fs

# 스케일로그램 및 ECG 세그먼트 플로팅 함수
def plot_segmented_beats(ecg_signal, fs, start_index, end_index):
    segment = ecg_signal[start_index:end_index]
    scales = np.arange(1, 64)
    wavelet_coeffs, freqs = pywt.cwt(segment, scales, wavelet, sampling_period=1/fs)
    
    # Set up the plot for the ECG segment and the scalogram
    plt.figure(figsize=(15, 3))
    
    # Plot the ECG segment
    plt.subplot(1, 3, 1)
    plt.plot(segment, label='ECG Signal')
    plt.title(f'ECG Segment ({start_index/fs:.2f}s to {end_index/fs:.2f}s)')
    plt.xlabel('Samples')
    plt.ylabel('Amplitude')
    plt.legend()
    
    # Plot the scalogram
    plt.subplot(1, 3, 2)
    plt.imshow((np.abs(wavelet_coeffs)), cmap='turbo', aspect='auto', origin='lower')
    plt.title('Scalogram')
    plt.xlabel('Time')
    plt.ylabel('Scales')
    
    # Plot the polar scalogram
    times = np.linspace(0, len(segment) / fs, len(segment)) 
    theta = 2 * np.pi * times / max(times) 
    rho = np.linspace(0.1, max(scales), len(scales))
    T, R = np.meshgrid(theta, rho)
    plt.subplot(1, 3, 3, polar=True)
    plt.pcolormesh(T, R, np.flipud(np.abs(wavelet_coeffs)), shading='auto', cmap='turbo')
    plt.title('Polar Scalogram')
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# 슬라이더를 이용한 인터랙티브 플롯 생성
def interactive_plot(ecg_signal, fs):
    max_index = len(ecg_signal)
    start_slider = IntSlider(min=0, max=max_index, step=int(0.5*fs), value=0, description='Start Index:')
    end_slider = IntSlider(min=0, max=max_index, step=int(0.5*fs), value=int(fs), description='End Index:')
    interactive_plot = interactive(plot_segmented_beats, ecg_signal=fixed(ecg_signal), fs=fixed(fs), start_index=start_slider, end_index=end_slider)
    return interactive_plot

# ECG 데이터와 R파 위치 로드 및 인터랙티브 플롯 표시
ecg_signal, fs = ecg_read('A00004')  # ECG 데이터를 로드하는 함수
interactive_ui = interactive_plot(ecg_signal, fs)
interactive_ui
